In [18]:
!pip install pillow

In [53]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data import DatasetMapper
from detectron2.model_zoo import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from PIL import Image
import os

In [111]:
# Step 1: Define the register_custom_dataset function
def register_custom_dataset(name, image_root):
    # Register the custom dataset with Detectron2
    DatasetCatalog.register("my_dataset_train", train_dataset_function)
    MetadataCatalog.get("my_dataset_train").set(thing_classes=["hat", "glasses"])


In [68]:
# Step 2: Unregister existing datasets and metadata if needed
if "my_dataset_train" in DatasetCatalog:
    DatasetCatalog.remove("my_dataset_train")
    MetadataCatalog.remove("my_dataset_train")
if "my_dataset_val" in DatasetCatalog:
    DatasetCatalog.remove("my_dataset_val")
    MetadataCatalog.remove("my_dataset_val")
if "my_dataset_test" in DatasetCatalog:
    DatasetCatalog.remove("my_dataset_test")
    MetadataCatalog.remove("my_dataset_test")

# Step 3: Register datasets
register_custom_dataset("my_dataset_train", "C:/Users/shane/object-detect-main/data_models/images/train")
register_custom_dataset("my_dataset_val", "C:/Users/shane/object-detect-main/data_models/images/val")
register_custom_dataset("my_dataset_test", "C:/Users/shane/object-detect-main/data_models/images/test")

In [104]:
class CustomDatasetMapper(DatasetMapper):
    def __init__(self, cfg, is_train=True):
        super().__init__(cfg, is_train)
        self.img_format = cfg.INPUT.FORMAT

    def __call__(self, dataset_dict):
        # Ensure dataset_dict contains the expected keys
        if isinstance(dataset_dict, dict):
            file_name = dataset_dict.get("file_name", None)
        elif isinstance(dataset_dict, (list, tuple)):
            file_name = dataset_dict[0] if len(dataset_dict) > 0 else None
        else:
            raise ValueError("Invalid dataset_dict format.")

        if file_name is None:
            raise ValueError("Invalid dataset_dict format. Missing 'file_name' key.")

        print("Attempting to open file:", file_name)  # Add this line for debugging
        try:
            image = utils.read_image(file_name, format=self.img_format)
        except Exception as e:
            print("Error opening file:", file_name)
            print(e)
            raise
        dataset_dict["image"] = image
        return dataset_dict

In [105]:
# Step 5: Choose a model configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

In [106]:
# Step 6: Set other training parameters
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)  # Optionally for evaluation
cfg.DATALOADER.NUM_WORKERS = 4  # Number of data loading threads
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 100
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get("my_dataset_train").thing_classes)

In [107]:
# Step 7: Set output directory where the trained model will be saved
cfg.OUTPUT_DIR = r"C:\Users\shane\object-detect-main\data_models\Detector_model"

# Step 8: Set device to CPU
cfg.MODEL.DEVICE = "cpu"

In [108]:
# Step 9: Create a trainer and train the model
trainer = DefaultTrainer(cfg)


[03/24 13:43:01 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[03/24 13:43:01 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[03/24 13:43:01 d2.data.build]: Using training sampler TrainingSampler
[03/24 13:43:01 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[03/24 13:43:01 d2.data.common]: Serializing 164 elements to byte tensors and concatenating them all ...
[03/24 13:43:01 d2.data.common]: Serialized dataset takes 0.02 MiB
[03/24 13:43:01 d2.data.build]: Making batched data loader with batch_size=2
WARNING [03/24 13:43:01 d2.solver.build]: SOLVER.STEPS contains values larger than SOLVER.MAX_ITER. These values will be ignored.


In [109]:
trainer.resume_or_load(resume=False)


[03/24 13:43:01 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (2, 256, 1, 1) in

In [110]:
trainer.train()

[03/24 13:43:03 d2.engine.train_loop]: Starting training from iteration 0
ERROR [03/24 13:43:07 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\detectron2\engine\train_loop.py", line 155, in train
    self.run_step()
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\detectron2\engine\defaults.py", line 496, in run_step
    self._trainer.run_step()
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\detectron2\engine\train_loop.py", line 297, in run_step
    data = next(self._data_loader_iter)
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\detectron2\data\common.py", line 329, in __iter__
    for d in self.dataset:
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\torch\utils\data\dataloader.py", line 631, in __next__
    data = self._next_data()
  File "C:\Users\shane\anaconda3\envs\Pyt

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\torch\utils\data\_utils\fetch.py", line 32, in fetch
    data.append(next(self.dataset_iter))
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\detectron2\data\common.py", line 296, in __iter__
    yield self.dataset[idx]
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\detectron2\data\common.py", line 125, in __getitem__
    data = self._map_func(self._dataset[cur_idx])
  File "C:\Users\shane\anaconda3\envs\Python3-10-Build\lib\site-packages\detectron2\data\dataset_mapper.py", line 154, in __call__
    image = utils.read_image(dataset_dict["file_name"], format=self.image_format)
TypeError: tuple indic